In [1]:
import json
import random
import time
import socket
import os.path
from os import path
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
DISCOUNT_RATE = 0.99
LEARNING_RATE = 0.01
BATCH_SIZE = 500
REPLAY_MEMORY_CAPACITY = 20000
TARGET_UPDATE_STEP = 1000
MAX_STEPS_PER_EPISODE = 5000
TOTAL_EPISODES = 10000
POLICY_NETWORK_SAVE_FILE = "./DrMarioDQN_final/policy_network"

In [4]:
loss_function = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)

In [5]:
class Communicator:
    """Provides a socket communication between the DQN and Dr.Mario"""
    DR_MARIO_HOST = '192.168.0.107'
    PORT = 42012
    BUFFER_SIZE = 1024
    __connection = None
    
    def __init__(self):
        self.__connection = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 

    def connect(self):
        self.__connection.connect((self.DR_MARIO_HOST, self.PORT))

    def read_state(self):
        state_str = self.__connection.recv(self.BUFFER_SIZE).decode()
        return state_str

    def send_action(self, action):
        self.__connection.sendall(str.encode(action))

    def __del__(self):
        self.__connection.close()

In [6]:
class State:
    board = None
    __position = None
    condition = None
    NUM_ROWS = 8
    NUM_COLS = 8
    STATE_SIZE = NUM_ROWS * NUM_COLS + 2
    TILE_TYPE_ENCODING = {
        "E": 0,
        "RV": 1, "YV": 2, "BV": 3,
        "RPL": 4, "RPR": 5, "RPU": 6, "RPD": 7, "RPN": 8,
        "YPL": 9, "YPR": 10, "YPU": 11, "YPD": 12, "YPN": 13,
        "BPL": 14, "BPR": 15, "BPU": 16, "BPD": 17, "BPN": 18,
    }

    def __init__(self, board, position, condition):
        self.board = board
        self.__position = position
        self.condition = condition
    
    def encodeState(self):
        encoded_state = self._encodeBoard()
        encoded_state.append(self.__position['row'])
        encoded_state.append(self.__position['col'])
        return encoded_state

    def _encodeBoard(self):
        encoded_board = []
        for row in self.board:
            for tile in row:
                encoded_board.append(self.TILE_TYPE_ENCODING[tile])
        
        return encoded_board

In [7]:
class Experience:
    state = None
    action = None
    reward = None
    next_state = None
    
    def __init__(self, state, action, reward, next_state):
        self.state = state
        self.action = action
        self.reward = reward
        self.next_state = next_state

In [8]:
class RewardManager:
    @staticmethod
    def getReward(state, next_state):
        if RewardManager.hasPlayerWon(next_state):
            return 100
        if RewardManager.hasPlayerLost(next_state):
            return -50

        state_viruses = RewardManager.__countViruses(state)
        next_state_viruses = RewardManager.__countViruses(next_state)

        reward = -1
        reward += RewardManager.getNumberNonEmptyTiles(state)
        reward -= RewardManager.getNumberNonEmptyTiles(next_state)
        reward += (state_viruses - next_state_viruses) * 10
        
        return reward

    @staticmethod
    def hasPlayerWon(state):
        return state.condition == "W"

    @staticmethod 
    def hasPlayerLost(state):
        return state.condition == "L"

    @staticmethod
    def getNumberNonEmptyTiles(state):
        number_non_empty_tiles = 0

        for row in state.board:
            for tile in row:
                if tile != "E":
                   number_non_empty_tiles += 1

        return number_non_empty_tiles

    @staticmethod
    def __countViruses(state):
        count = 0
        for row in state.board:
            for tile in row:
                if len(tile) == 2:
                    count += 1
        return count

In [9]:
class Environment:
    RESET_ACTION = "RS"
    QUIT_ACTION = "QT"
    PLAYER_ACTIONS = ["ML", "MR", "RL", "RR", "WT"]
    __communicator = None
    __current_state = None

    def __init__(self):
        self.__communicator = Communicator()
        self.__communicator.connect()
        self.reset()

    def reset(self):
        self.__communicator.send_action(self.RESET_ACTION)
        self.__current_state = self.__getState()
        return self.getCurrentState().encodeState()
        
    def getCurrentState(self):
        return self.__current_state

    def quit(self):
        self.__communicator.send_action(self.QUIT_ACTION)
    
    def makeAction(self, action):
        state = self.__current_state

        self.__communicator.send_action(self.PLAYER_ACTIONS[action])
        next_state = self.__getState()

        reward = RewardManager.getReward(self.__current_state, next_state)

        self.__current_state = next_state
        return Experience(state.encodeState(), action, reward, next_state.encodeState())

    def hasGameEnded(self):
        return self.__current_state.condition != "O"

    def __getState(self):
        state_str = self.__communicator.read_state()
        state_json = json.loads(state_str)
        return State(state_json["board"], state_json["position"], state_json["condition"])

In [10]:
class ReplayMemory:
    memory = []
    __capacity = 0
    __position_to_add = 0

    def __init__(self, capacity):
        self.__capacity = capacity

    def add(self, experience):
        if not(self.__isFull()):
            self.memory.append(experience)
        else:
            self.memory[self.__position_to_add] = experience
            self.__position_to_add = (self.__position_to_add + 1) % self.__capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def canProvideBatch(self, batch_size):
        return len(self.memory) >= batch_size
    
    def __isFull(self):
        return len(self.memory) == self.__capacity


In [11]:
def create_model():
    inputs = layers.Input(shape=(State.STATE_SIZE,))

    layer1 = layers.Dense(256, activation="relu")(inputs)
    layer2 = layers.Dense(128, activation="relu")(layer1)
    action = layers.Dense(len(Environment.PLAYER_ACTIONS), activation="linear")(layer2)

    return keras.Model(inputs=inputs, outputs=action)

In [12]:
class Agent:
    EPSILON_MIN = 0.01
    EPSILON_MAX = 1.0
    EPSILON_DECAY_RATE = 0.0025

    epsilon = None
    
    actions_count = 0
    model = None
    target_model = None

    def __init__(self, actions_count):
        self.actions_count = actions_count
        self.model = create_model()
        if path.exists(POLICY_NETWORK_SAVE_FILE + ".index"):
            self.model.load_weights(POLICY_NETWORK_SAVE_FILE)
        self.target_model = create_model()
        self.updateTargetModel()
        self.epsilon = self.EPSILON_MAX

    def updateTargetModel(self):
        self.target_model.set_weights(self.model.get_weights())
    
    def __explore(self):
        return random.randrange(self.actions_count)

    def __exploit(self, state):
        state_tensor = tf.convert_to_tensor(state)
        state_tensor = tf.expand_dims(state_tensor, 0)
        action_probs = self.model(state_tensor, training=False)
        action = tf.argmax(action_probs[0]).numpy()
        return action

    def chooseAction(self, state):
        if random.random() > self.epsilon:
            return self.__exploit(state)
        else:
            return self.__explore()

    def decayEpsilon(self, episode):
        self.epsilon = self.EPSILON_MIN + (self.EPSILON_MAX - self.EPSILON_MIN) * np.exp(-episode * self.EPSILON_DECAY_RATE)
        

In [13]:
def disassembleBatch(experiences):
    states = []
    actions = []
    rewards = []
    next_states = []

    for experience in experiences:
        states.append(experience.state)
        actions.append(experience.action)
        rewards.append(experience.reward)
        next_states.append(experience.next_state)

    return (np.array(states), np.array(actions), np.array(rewards), np.array(next_states))


In [14]:
episodes_rewards = []
memory = ReplayMemory(REPLAY_MEMORY_CAPACITY)

In [15]:
env = Environment()
agent = Agent(len(env.PLAYER_ACTIONS))
learning_steps = 0

In [ ]:
number_steps_made = 0

for episode in range(0, TOTAL_EPISODES):
    agent.model.save_weights(POLICY_NETWORK_SAVE_FILE)
    state = env.reset()
    episode_reward = 0
    number_steps_made_in_episode = 0

    print(f'episode number: {episode}')
    print(f'episodes rewards: {episodes_rewards}')
    
    for step in range(0, MAX_STEPS_PER_EPISODE):
        number_steps_made += 1
        number_steps_made_in_episode += 1
        action = agent.chooseAction(state)
        experience = env.makeAction(action)
        memory.add(experience)
        state = experience.next_state
        episode_reward += experience.reward

        if memory.canProvideBatch(BATCH_SIZE):
            learning_steps += 1
            batch = memory.sample(BATCH_SIZE)
            states, actions, rewards, next_states = disassembleBatch(batch)

            opt_q_next_state = agent.target_model.predict(next_states)
            expected_opt_q = rewards + DISCOUNT_RATE * tf.reduce_max(
                opt_q_next_state, axis=1
            )
            masks = tf.one_hot(actions, agent.actions_count)

            with tf.GradientTape() as tape:
                q_values = agent.model(states)
                q_for_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                loss = loss_function(expected_opt_q, q_for_action)

            # Backpropagation
            grads = tape.gradient(loss, agent.model.trainable_variables)
            optimizer.apply_gradients(zip(grads, agent.model.trainable_variables))

        if (learning_steps != 0) and (learning_steps % TARGET_UPDATE_STEP == 0):
            agent.updateTargetModel()

        if env.hasGameEnded():
            break
    
    print(f'number steps made: {number_steps_made}')
    print(f'number steps made in episode: {number_steps_made_in_episode}')
    print(f'epsilon: {agent.epsilon}')
    print()
    agent.decayEpsilon(episode)
    episodes_rewards.append(episode_reward)

print(episodes_rewards)
env.quit()

episode number: 0
episodes rewards: []
number steps made: 225
number steps made in episode: 225
epsilon: 1.0

episode number: 1
episodes rewards: [-296]
number steps made: 510
number steps made in episode: 285
epsilon: 1.0

episode number: 2
episodes rewards: [-296, -352]
number steps made: 1314
number steps made in episode: 804
epsilon: 0.9975280911734855

episode number: 3
episodes rewards: [-296, -352, -879]
number steps made: 1721
number steps made in episode: 407
epsilon: 0.9950623544007555

episode number: 4
episodes rewards: [-296, -352, -879, -484]
number steps made: 1874
number steps made in episode: 153
epsilon: 0.992602774270947

episode number: 5
episodes rewards: [-296, -352, -879, -484, -214]
number steps made: 2001
number steps made in episode: 127
epsilon: 0.9901493354116764

episode number: 6
episodes rewards: [-296, -352, -879, -484, -214, -186]
number steps made: 2493
number steps made in episode: 492
epsilon: 0.9877020224889427

episode number: 7
episodes rewards: [

number steps made: 14066
number steps made in episode: 353
epsilon: 0.9170566829343688

episode number: 37
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505, -372, -321, -256, -689, -581, -378, -520, -409]
number steps made: 14338
number steps made in episode: 272
epsilon: 0.9147918734185159

episode number: 38
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505, -372, -321, -256, -689, -581, -378, -520, -409, -339]
number steps made: 14650
number steps made in episode: 312
epsilon: 0.9125327188548171

episode number: 39
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505

number steps made: 21294
number steps made in episode: 252
epsilon: 0.8706646530448178

episode number: 58
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505, -372, -321, -256, -689, -581, -378, -520, -409, -339, -381, -388, -330, -424, -408, -476, -285, -631, -533, -268, -198, -318, -590, -452, -259, -457, -656, -551, -383, -321]
number steps made: 21634
number steps made in episode: 340
epsilon: 0.8685156787493323

episode number: 59
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505, -372, -321, -256, -689, -581, -378, -520, -409, -339, -381, -388, -330, -424, -408, -476, -285, -631, -533, -268, -198, -318, -590, -452, -259, -457, -656, -551, -383, -321, -407]
number steps made: 21704
number steps made in 

number steps made: 25589
number steps made in episode: 332
epsilon: 0.8369175092971592

episode number: 74
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505, -372, -321, -256, -689, -581, -378, -520, -409, -339, -381, -388, -330, -424, -408, -476, -285, -631, -533, -268, -198, -318, -590, -452, -259, -457, -656, -551, -383, -321, -407, -127, -431, -406, -401, -267, -218, -231, -373, -273, -211, -631, -204, -198, -594, -399]
number steps made: 25722
number steps made in episode: 133
epsilon: 0.8348527974890472

episode number: 75
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505, -372, -321, -256, -689, -581, -378, -520, -409, -339, -381, -388, -330, -424, -408, -476, -285, -631, -533, -268, -198, -318, -590

number steps made: 30498
number steps made in episode: 405
epsilon: 0.8084760257755536

episode number: 88
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505, -372, -321, -256, -689, -581, -378, -520, -409, -339, -381, -388, -330, -424, -408, -476, -285, -631, -533, -268, -198, -318, -590, -452, -259, -457, -656, -551, -383, -321, -407, -127, -431, -406, -401, -267, -218, -231, -373, -273, -211, -631, -204, -198, -594, -399, -194, -586, -344, -513, -580, -509, -254, -387, -455, -419, -381, -337, -471, -468]
number steps made: 30802
number steps made in episode: 304
epsilon: 0.8064823288706297

episode number: 89
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505, -372, -321, -256, -689, -581, -378, -520, -409

...

number steps made: 152077
number steps made in episode: 287
epsilon: 0.2666478580394326

episode number: 542
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505, -372, -321, -256, -689, -581, -378, -520, -409, -339, -381, -388, -330, -424, -408, -476, -285, -631, -533, -268, -198, -318, -590, -452, -259, -457, -656, -551, -383, -321, -407, -127, -431, -406, -401, -267, -218, -231, -373, -273, -211, -631, -204, -198, -594, -399, -194, -586, -344, -513, -580, -509, -254, -387, -455, -419, -381, -337, -471, -468, -371, -417, -271, -348, -427, -320, -352, -328, -243, -281, -227, -358, -289, -501, -356, -413, -376, -425, -560, -298, -542, -632, -344, -350, -293, -503, -587, -397, -503, -316, -332, -683, -304, -430, -261, -422, -351, -562, -225, -337, -203, -357, -374, -398, -518, -297, -348, -245, -495, -243, -137, -474, -716, -178, -180, -368, -131, -1

number steps made: 153279
number steps made in episode: 496
epsilon: 0.26473019931337644

episode number: 545
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505, -372, -321, -256, -689, -581, -378, -520, -409, -339, -381, -388, -330, -424, -408, -476, -285, -631, -533, -268, -198, -318, -590, -452, -259, -457, -656, -551, -383, -321, -407, -127, -431, -406, -401, -267, -218, -231, -373, -273, -211, -631, -204, -198, -594, -399, -194, -586, -344, -513, -580, -509, -254, -387, -455, -419, -381, -337, -471, -468, -371, -417, -271, -348, -427, -320, -352, -328, -243, -281, -227, -358, -289, -501, -356, -413, -376, -425, -560, -298, -542, -632, -344, -350, -293, -503, -587, -397, -503, -316, -332, -683, -304, -430, -261, -422, -351, -562, -225, -337, -203, -357, -374, -398, -518, -297, -348, -245, -495, -243, -137, -474, -716, -178, -180, -368, -131, -

number steps made: 153953
number steps made in episode: 152
epsilon: 0.2628268692281969

episode number: 548
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505, -372, -321, -256, -689, -581, -378, -520, -409, -339, -381, -388, -330, -424, -408, -476, -285, -631, -533, -268, -198, -318, -590, -452, -259, -457, -656, -551, -383, -321, -407, -127, -431, -406, -401, -267, -218, -231, -373, -273, -211, -631, -204, -198, -594, -399, -194, -586, -344, -513, -580, -509, -254, -387, -455, -419, -381, -337, -471, -468, -371, -417, -271, -348, -427, -320, -352, -328, -243, -281, -227, -358, -289, -501, -356, -413, -376, -425, -560, -298, -542, -632, -344, -350, -293, -503, -587, -397, -503, -316, -332, -683, -304, -430, -261, -422, -351, -562, -225, -337, -203, -357, -374, -398, -518, -297, -348, -245, -495, -243, -137, -474, -716, -178, -180, -368, -131, -1

number steps made: 155186
number steps made in episode: 181
epsilon: 0.26093776072107494

episode number: 551
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505, -372, -321, -256, -689, -581, -378, -520, -409, -339, -381, -388, -330, -424, -408, -476, -285, -631, -533, -268, -198, -318, -590, -452, -259, -457, -656, -551, -383, -321, -407, -127, -431, -406, -401, -267, -218, -231, -373, -273, -211, -631, -204, -198, -594, -399, -194, -586, -344, -513, -580, -509, -254, -387, -455, -419, -381, -337, -471, -468, -371, -417, -271, -348, -427, -320, -352, -328, -243, -281, -227, -358, -289, -501, -356, -413, -376, -425, -560, -298, -542, -632, -344, -350, -293, -503, -587, -397, -503, -316, -332, -683, -304, -430, -261, -422, -351, -562, -225, -337, -203, -357, -374, -398, -518, -297, -348, -245, -495, -243, -137, -474, -716, -178, -180, -368, -131, -

number steps made: 155786
number steps made in episode: 259
epsilon: 0.2590627675291589

episode number: 554
episodes rewards: [-296, -352, -879, -484, -214, -186, -576, -459, -625, -788, -309, -530, -739, -426, -266, -516, -489, -185, -243, -374, -643, -629, -226, -436, -431, -438, -389, -489, -505, -372, -321, -256, -689, -581, -378, -520, -409, -339, -381, -388, -330, -424, -408, -476, -285, -631, -533, -268, -198, -318, -590, -452, -259, -457, -656, -551, -383, -321, -407, -127, -431, -406, -401, -267, -218, -231, -373, -273, -211, -631, -204, -198, -594, -399, -194, -586, -344, -513, -580, -509, -254, -387, -455, -419, -381, -337, -471, -468, -371, -417, -271, -348, -427, -320, -352, -328, -243, -281, -227, -358, -289, -501, -356, -413, -376, -425, -560, -298, -542, -632, -344, -350, -293, -503, -587, -397, -503, -316, -332, -683, -304, -430, -261, -422, -351, -562, -225, -337, -203, -357, -374, -398, -518, -297, -348, -245, -495, -243, -137, -474, -716, -178, -180, -368, -131, -1